<h1> Featured Activity 2: Convert and Transform Raw Data: European Soccer Data </h1>

Import the required libraries.

In [1]:
library(dplyr)
library(purrr)
library(tidyr)
library(ggplot2)
library(broom)
library(magrittr)
library(RSQLite)
library(reshape2)
library(jsonlite)
library(knitr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'purrr'

The following objects are masked from 'package:dplyr':

    contains, order_by


Attaching package: 'magrittr'

The following object is masked from 'package:tidyr':

    extract

The following object is masked from 'package:purrr':

    set_names


Attaching package: 'reshape2'

The following object is masked from 'package:tidyr':

    smiths


Attaching package: 'jsonlite'

The following object is masked from 'package:purrr':

    flatten



Get the data from tables in data frames.

In [2]:
library(data.table)
con = dbConnect(SQLite(), dbname="soccerdatabase.sqlite")
player = tbl_df(dbGetQuery(con,"SELECT * FROM player"))
Match = tbl_df(dbGetQuery(con,"SELECT * FROM Match"))
Team = tbl_df(dbGetQuery(con,"SELECT * FROM Team"))
Country = tbl_df(dbGetQuery(con,"SELECT * FROM Country"))
League = tbl_df(dbGetQuery(con,"SELECT * FROM League"))
PlayerAttributes <- tbl_df(dbGetQuery(con,"SELECT * FROM Player_Attributes"))
TeamAttributes <- tbl_df(dbGetQuery(con,"SELECT * FROM Team_Attributes"))

------------------------------------------------------------------------------
data.table + dplyr code now lives in dtplyr.
Please library(dtplyr)!
------------------------------------------------------------------------------

Attaching package: 'data.table'

The following objects are masked from 'package:reshape2':

    dcast, melt

The following object is masked from 'package:purrr':

    transpose

The following objects are masked from 'package:dplyr':

    between, first, last



Selecting the requied columns.

In [3]:
player = select(player,player_api_id, player_name) # use player_api_id as key for join
Team = select(Team, team_api_id, team_long_name, team_short_name) # use team_api_id as key for join
Country = select(Country, id, name) %>% rename(country_id = id)  %>% rename(country_name = name)   # use country_id as key for join
League = select(League, country_id, name) %>% rename(league_name = name) # use country_id as key for join
Match = select(Match, id, country_id, league_id, season, stage, date, match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7, home_player_8, home_player_9, home_player_10, home_player_11, away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7, away_player_8, away_player_9, away_player_10, away_player_11, goal, shoton, shotoff, foulcommit, card, cross, corner, possession)


In order to answer all the questions, using melt function and join to get a data frame including all the required columns. Removing NAs from the table.

In [4]:
matchMelt <-melt(Match,id = c(1:11), measure=c(12:33),na.rm = TRUE, value.name = "player_api_id") %>% 
  mutate(team_api_id=ifelse(grepl("home",variable),home_team_api_id,
                            ifelse(grepl("away",variable),away_team_api_id,NA))) %>%  # create team_api_id column based on variable info
  left_join(Team, by = "team_api_id") %>%
  left_join(player, by = "player_api_id") %>%
  left_join(Country, by = "country_id") %>% 
  left_join(League, by = "country_id") 


In [5]:
collect(matchMelt)

id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,variable,player_api_id,team_api_id,team_long_name,team_short_name,player_name,country_name,league_name
145,1,1,2008/2009,24,2009-02-28 00:00:00,493016,9996,8635,1,1,home_player_1,39890,9996,Royal Excel Mouscron,MOU,Mark Volders,Belgium,Belgium Jupiler League
146,1,1,2008/2009,24,2009-02-27 00:00:00,493017,8203,9987,2,1,home_player_1,38327,8203,KV Mechelen,MEC,Wouter Biebauw,Belgium,Belgium Jupiler League
147,1,1,2008/2009,24,2009-02-28 00:00:00,493018,9986,9998,3,0,home_player_1,95597,9986,Sporting Charleroi,CHA,Bertrand Laquait,Belgium,Belgium Jupiler League
149,1,1,2008/2009,24,2009-03-01 00:00:00,493020,9994,9991,0,1,home_player_1,30934,9994,Sporting Lokeren,LOK,Boubacar Barry Copa,Belgium,Belgium Jupiler League
150,1,1,2008/2009,24,2009-02-28 00:00:00,493021,8342,9999,2,1,home_player_1,37990,8342,Club Brugge KV,CLB,Stijn Stijnen,Belgium,Belgium Jupiler League
151,1,1,2008/2009,24,2009-02-28 00:00:00,493022,9993,8571,3,0,home_player_1,38391,9993,Beerschot AC,BAC,Silvio Proto,Belgium,Belgium Jupiler League
152,1,1,2008/2009,24,2009-02-28 00:00:00,493023,7947,4049,4,0,home_player_1,39153,7947,FCV Dender EH,DEN,Cedric Berthelin,Belgium,Belgium Jupiler League
153,1,1,2008/2009,24,2009-02-28 00:00:00,493024,10000,10001,1,1,home_player_1,37900,10000,SV Zulte-Waregem,ZUL,Sammy Bossuyt,Belgium,Belgium Jupiler League
154,1,1,2008/2009,25,2009-03-08 00:00:00,493025,9984,8342,1,3,home_player_1,36835,9984,KSV Cercle Brugge,CEB,Bram Verbist,Belgium,Belgium Jupiler League
155,1,1,2008/2009,25,2009-03-06 00:00:00,493026,9998,9985,0,1,home_player_1,38252,9998,RAEC Mons,MON,Frederic Herpoel,Belgium,Belgium Jupiler League


Getting players team-wise.

In [6]:
temptb=summarize((group_by(matchMelt, player_api_id,team_api_id)))
temptb

player_api_id,team_api_id
2625,2183
2625,8021
2625,8569
2752,7878
2768,8370
2768,8603
2768,10205
2770,8581
2770,9985
2790,10281


<b> Question :  Which are the top 10 teams by number of fielded players (i.e. number of players the team has sent to the field across all matches)?</b>

Finding number of players from a team to get fielded players

In [7]:
teamFieldedPlayers=summarize(group_by(temptb,team_api_id),noOfPlayers=n())
teamFieldedPlayers

team_api_id,noOfPlayers
1601,55
1773,55
1957,64
2033,87
2182,60
2183,35
2186,49
4049,16
4064,27
4087,56


Finding number of teams of each player.

In [8]:
tempTeamCount=summarize(group_by(temptb,player_api_id),noOfTeams=n())

In [9]:
tempTeamCount

player_api_id,noOfTeams
2625,3
2752,1
2768,3
2770,2
2790,1
2796,1
2802,3
2805,5
2857,1
2862,1


Appending the number of teams column to our player table. 

In [10]:
player=left_join(player,tempTeamCount,by="player_api_id")

In [11]:
player

player_api_id,player_name,noOfTeams
505942,Aaron Appindangoye,1
155782,Aaron Cresswell,1
162549,Aaron Doran,1
30572,Aaron Galindo,1
23780,Aaron Hughes,1
27316,Aaron Hunt,3
564793,Aaron Kuhl,1
30895,Aaron Lennon,2
528212,Aaron Lennox,1
101042,Aaron Meijers,3


<b>Q. Which are the top 10 players by number of appearances (with any team they played with)?</b>

In [12]:
top10appearance=summarize(group_by(matchMelt,player_api_id),totalMatchPlayed=n())

In [13]:
top10appearance

player_api_id,totalMatchPlayed
2625,90
2752,81
2768,71
2770,16
2790,50
2796,1
2802,144
2805,131
2857,3
2862,17


In [14]:
player=left_join(player,top10appearance,by="player_api_id")
player

player_api_id,player_name,noOfTeams,totalMatchPlayed
505942,Aaron Appindangoye,1,8
155782,Aaron Cresswell,1,75
162549,Aaron Doran,1,104
30572,Aaron Galindo,1,10
23780,Aaron Hughes,1,162
27316,Aaron Hunt,3,158
564793,Aaron Kuhl,1,5
30895,Aaron Lennon,2,181
528212,Aaron Lennox,1,1
101042,Aaron Meijers,3,167


<b> Q: Which are the top 10 players by number of leagues they played in? </b>


Grouping the player id and league_name.

In [15]:
tempLeaguetb=summarize((group_by(matchMelt, player_api_id,league_name)))
tempLeaguetb

player_api_id,league_name
2625,Poland Ekstraklasa
2752,Spain LIGA BBVA
2768,Spain LIGA BBVA
2770,Belgium Jupiler League
2770,Spain LIGA BBVA
2790,Spain LIGA BBVA
2796,Spain LIGA BBVA
2802,England Premier League
2802,Spain LIGA BBVA
2805,Spain LIGA BBVA


Counting the number of leagues of each player by using summarize function.

In [16]:
tempLeagueCount=summarize(group_by(tempLeaguetb,player_api_id),noOfLeagues=n())

In [17]:
tempLeagueCount=arrange(tempLeagueCount,desc(noOfLeagues))
tempLeagueCount

player_api_id,noOfLeagues
42119,6
116750,5
178538,5
22543,4
24123,4
25816,4
26434,4
26489,4
26675,4
30477,4


Appending the number of leagues column to our player table. 

In [18]:
player=left_join(player,tempLeagueCount,by="player_api_id")

In [19]:
player

player_api_id,player_name,noOfTeams,totalMatchPlayed,noOfLeagues
505942,Aaron Appindangoye,1,8,1
155782,Aaron Cresswell,1,75,1
162549,Aaron Doran,1,104,1
30572,Aaron Galindo,1,10,1
23780,Aaron Hughes,1,162,1
27316,Aaron Hunt,3,158,1
564793,Aaron Kuhl,1,5,1
30895,Aaron Lennon,2,181,1
528212,Aaron Lennox,1,1,1
101042,Aaron Meijers,3,167,1


<b>Q: Which are the top 10 players by overall rating?</b>

selecting the required columns from player attributes table.

In [20]:
playerAttr=select(PlayerAttributes,player_api_id,overall_rating)
playerAttr

player_api_id,overall_rating
505942,67
505942,67
505942,62
505942,61
505942,61
155782,74
155782,74
155782,73
155782,73
155782,73


grouping it by player api id and then if multiple entires, retrieving the maximum of it.

In [21]:
playerAttr=summarize(group_by(playerAttr,player_api_id),rating=max(overall_rating))
playerAttr

player_api_id,rating
2625,63
2752,72
2768,75
2770,74
2790,73
2796,69
2802,82
2805,76
2857,72
2862,76


Appending the rating column to our player table. 

In [22]:
player=left_join(player,playerAttr,by="player_api_id")

In [23]:
player

player_api_id,player_name,noOfTeams,totalMatchPlayed,noOfLeagues,rating
505942,Aaron Appindangoye,1,8,1,67
155782,Aaron Cresswell,1,75,1,74
162549,Aaron Doran,1,104,1,71
30572,Aaron Galindo,1,10,1,75
23780,Aaron Hughes,1,162,1,78
27316,Aaron Hunt,3,158,1,79
564793,Aaron Kuhl,1,5,1,61
30895,Aaron Lennon,2,181,1,84
528212,Aaron Lennox,1,1,1,48
101042,Aaron Meijers,3,167,1,69


<b>Q. Which are the top 10 goalkeepers by sum of gk attributes? </b>

selecting the required columns of gk attributes.

In [24]:
playerGKAttr=select(PlayerAttributes, player_api_id,starts_with("gk_"))

In [25]:
playerGKAttr

player_api_id,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
505942,6,11,10,8,8
505942,6,11,10,8,8
505942,6,11,10,8,8
505942,5,10,9,7,7
505942,5,10,9,7,7
155782,14,7,9,9,12
155782,14,7,9,9,12
155782,14,7,9,9,12
155782,14,7,9,9,12
155782,14,7,9,9,12


Calculating the sum of gk attributes and storing it in new column called sum_gk.

In [26]:
playerGKAttr=mutate(playerGKAttr,sum_gk=rowSums(playerGKAttr[,c(2,3,4,5,6)]))
playerGKAttr

player_api_id,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,sum_gk
505942,6,11,10,8,8,43
505942,6,11,10,8,8,43
505942,6,11,10,8,8,43
505942,5,10,9,7,7,38
505942,5,10,9,7,7,38
155782,14,7,9,9,12,51
155782,14,7,9,9,12,51
155782,14,7,9,9,12,51
155782,14,7,9,9,12,51
155782,14,7,9,9,12,51


Grouping it with the player ids and then retrieving the maximum of them.

In [27]:
playerGKAttr=summarize(group_by(playerGKAttr,player_api_id),sum_gk=max(sum_gk))
playerGKAttr

player_api_id,sum_gk
2625,153
2752,53
2768,128
2770,148
2790,144
2796,340
2802,146
2805,141
2857,143
2862,146


Performing left join and adding sum_gk column to our player table.

In [28]:
player=left_join(player,playerGKAttr,by="player_api_id")
player

player_api_id,player_name,noOfTeams,totalMatchPlayed,noOfLeagues,rating,sum_gk
505942,Aaron Appindangoye,1,8,1,67,43
155782,Aaron Cresswell,1,75,1,74,121
162549,Aaron Doran,1,104,1,71,125
30572,Aaron Galindo,1,10,1,75,111
23780,Aaron Hughes,1,162,1,78,129
27316,Aaron Hunt,3,158,1,79,132
564793,Aaron Kuhl,1,5,1,61,64
30895,Aaron Lennon,2,181,1,84,146
528212,Aaron Lennox,1,1,1,48,237
101042,Aaron Meijers,3,167,1,69,54


Writing the player table to player.csv which contains required information to answer our questions.

In [29]:
write.csv(player, "player.csv",na="")

Selecting required columns and storing it accordingly to data frames.

In [30]:
buildTb=select(TeamAttributes, team_api_id,starts_with("build"))
chanceTb=select(TeamAttributes,team_api_id,starts_with("chance"))
defenceTb=select(TeamAttributes,team_api_id,starts_with("defence"))

In [31]:
buildTb

team_api_id,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass
9930,60,Balanced,NA,Little,50,Mixed,Organised
9930,52,Balanced,48,Normal,56,Mixed,Organised
9930,47,Balanced,41,Normal,54,Mixed,Organised
8485,70,Fast,NA,Little,70,Long,Organised
8485,47,Balanced,NA,Little,52,Mixed,Organised
8485,58,Balanced,NA,Little,62,Mixed,Organised
8485,62,Balanced,NA,Little,45,Mixed,Organised
8485,58,Balanced,64,Normal,62,Mixed,Organised
8485,59,Balanced,64,Normal,53,Mixed,Organised
8576,60,Balanced,NA,Little,40,Mixed,Organised


Replacing NAs with 0

In [32]:
buildTb[ is.na(buildTb) ] <- 0
chanceTb[ is.na(chanceTb) ] <- 0
defenceTb[ is.na(defenceTb) ] <- 0

<b>Q: Which are the top 10 teams by sum of build up play attributes?</b>

calculating the sum of build attributes and storing it new column.

In [33]:
buildTb=mutate(buildTb,sum_build=rowSums(buildTb[,c(2,4,6)]))
buildTb

team_api_id,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,sum_build
9930,60,Balanced,0,Little,50,Mixed,Organised,110
9930,52,Balanced,48,Normal,56,Mixed,Organised,156
9930,47,Balanced,41,Normal,54,Mixed,Organised,142
8485,70,Fast,0,Little,70,Long,Organised,140
8485,47,Balanced,0,Little,52,Mixed,Organised,99
8485,58,Balanced,0,Little,62,Mixed,Organised,120
8485,62,Balanced,0,Little,45,Mixed,Organised,107
8485,58,Balanced,64,Normal,62,Mixed,Organised,184
8485,59,Balanced,64,Normal,53,Mixed,Organised,176
8576,60,Balanced,0,Little,40,Mixed,Organised,100


In case of multiple entries, selecting the maximum of sum.

In [34]:
teamBuildAttr=summarize(group_by(buildTb,team_api_id),sum_build=max(sum_build))
teamBuildAttr

team_api_id,sum_build
1601,139
1773,153
1957,158
2033,147
2182,176
2183,80
2186,125
4087,156
4170,86
6269,179


Appending the sum_build to our team table.

In [35]:
Team=left_join(Team,teamBuildAttr,by="team_api_id")
Team

team_api_id,team_long_name,team_short_name,sum_build
9987,KRC Genk,GEN,148
9993,Beerschot AC,BAC,110
10000,SV Zulte-Waregem,ZUL,147
9994,Sporting Lokeren,LOK,159
9984,KSV Cercle Brugge,CEB,145
8635,RSC Anderlecht,AND,149
9991,KAA Gent,GEN,142
9998,RAEC Mons,MON,160
7947,FCV Dender EH,DEN,NA
9985,Standard de Liège,STL,161


<b>Q: Which are the top 10 teams by sum of chance creation attributes?</b>

In [36]:
chanceTb

team_api_id,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass
9930,60,Normal,65,Normal,55,Normal,Organised
9930,54,Normal,63,Normal,64,Normal,Organised
9930,54,Normal,63,Normal,64,Normal,Organised
8485,70,Risky,70,Lots,70,Lots,Organised
8485,53,Normal,48,Normal,52,Normal,Organised
8485,45,Normal,70,Lots,55,Normal,Organised
8485,40,Normal,50,Normal,55,Normal,Organised
8485,56,Normal,68,Lots,57,Normal,Organised
8485,51,Normal,72,Lots,63,Normal,Free Form
8576,45,Normal,35,Normal,55,Normal,Organised


Calculating the sum of chance columns and incase of multiple entries assigning the maximum of sum to the team.

In [37]:
chanceTb=mutate(chanceTb,sum_chance=rowSums(chanceTb[,c(2,4,6)]))
teamChanceAttr=summarize(group_by(chanceTb,team_api_id),sum_chance=max(sum_chance))
teamChanceAttr

team_api_id,sum_chance
1601,186
1773,157
1957,163
2033,165
2182,196
2183,180
2186,168
4087,180
4170,165
6269,172


Performing left join and adding sum_chance column to our team table.

In [38]:
Team=left_join(Team,teamChanceAttr,by="team_api_id")
Team

team_api_id,team_long_name,team_short_name,sum_build,sum_chance
9987,KRC Genk,GEN,148,182
9993,Beerschot AC,BAC,110,185
10000,SV Zulte-Waregem,ZUL,147,153
9994,Sporting Lokeren,LOK,159,154
9984,KSV Cercle Brugge,CEB,145,200
8635,RSC Anderlecht,AND,149,182
9991,KAA Gent,GEN,142,184
9998,RAEC Mons,MON,160,168
7947,FCV Dender EH,DEN,NA,NA
9985,Standard de Liège,STL,161,182


<b>Q: Which are the top 10 teams by sum of defence attributes?</b>

In [39]:
defenceTb

team_api_id,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
9930,50,Medium,55,Press,45,Normal,Cover
9930,47,Medium,44,Press,54,Normal,Cover
9930,47,Medium,44,Press,54,Normal,Cover
8485,60,Medium,70,Double,70,Wide,Cover
8485,47,Medium,47,Press,52,Normal,Cover
8485,40,Medium,40,Press,60,Normal,Cover
8485,42,Medium,42,Press,60,Normal,Cover
8485,41,Medium,42,Press,60,Normal,Cover
8485,49,Medium,45,Press,63,Normal,Cover
8576,30,Deep,70,Double,30,Narrow,Offside Trap


Calculating the sum of defence columns and incase of multiple entries assigning the maximum of sum to the team.

In [40]:
defenceTb=mutate(defenceTb,sum_defence=rowSums(defenceTb[,c(2,4,6)]))
teamDefenceAttr=summarize(group_by(defenceTb,team_api_id),sum_defence=max(sum_defence))
teamDefenceAttr

team_api_id,sum_defence
1601,175
1773,137
1957,210
2033,155
2182,160
2183,175
2186,205
4087,165
4170,200
6269,151


Performing left join and adding sum_defence column to our team table.

In [41]:
Team=left_join(Team,teamDefenceAttr,by="team_api_id")
Team

team_api_id,team_long_name,team_short_name,sum_build,sum_chance,sum_defence
9987,KRC Genk,GEN,148,182,205
9993,Beerschot AC,BAC,110,185,210
10000,SV Zulte-Waregem,ZUL,147,153,200
9994,Sporting Lokeren,LOK,159,154,200
9984,KSV Cercle Brugge,CEB,145,200,195
8635,RSC Anderlecht,AND,149,182,190
9991,KAA Gent,GEN,142,184,166
9998,RAEC Mons,MON,160,168,138
7947,FCV Dender EH,DEN,NA,NA,NA
9985,Standard de Liège,STL,161,182,205


In [42]:
Team=left_join(Team,teamFieldedPlayers,by="team_api_id")
Team

team_api_id,team_long_name,team_short_name,sum_build,sum_chance,sum_defence,noOfPlayers
9987,KRC Genk,GEN,148,182,205,75
9993,Beerschot AC,BAC,110,185,210,72
10000,SV Zulte-Waregem,ZUL,147,153,200,87
9994,Sporting Lokeren,LOK,159,154,200,70
9984,KSV Cercle Brugge,CEB,145,200,195,64
8635,RSC Anderlecht,AND,149,182,190,76
9991,KAA Gent,GEN,142,184,166,88
9998,RAEC Mons,MON,160,168,138,46
7947,FCV Dender EH,DEN,NA,NA,NA,13
9985,Standard de Liège,STL,161,182,205,114


<b>Q: Which are the top 10 teams by number of scored goals? </b>

1. Finding total goals made if home_team
2. Finding total goals made if away_team
3. Merge both them and then summarize with team_Api_id

In [43]:
homeTeamGoal=select(Match,home_team_api_id,home_team_goal)
homeTeamGoal=summarize(group_by(homeTeamGoal,home_team_api_id),totalGoals=sum(home_team_goal))
homeTeamGoal=rename(homeTeamGoal,team_api_id=home_team_api_id) #rename column for rbind
homeTeamGoal
awayTeamGoal=select(Match,away_team_api_id,away_team_goal)
awayTeamGoal=summarize(group_by(awayTeamGoal,away_team_api_id),totalGoals=sum(away_team_goal))
awayTeamGoal=rename(awayTeamGoal,team_api_id=away_team_api_id)
awayTeamGoal
totalgoals=rbind(homeTeamGoal,awayTeamGoal)

team_api_id,totalGoals
1601,148
1773,74
1957,176
2033,70
2182,215
2183,96
2186,144
4049,22
4064,13
4087,105


team_api_id,totalGoals
1601,132
1773,52
1957,111
2033,68
2182,157
2183,88
2186,91
4049,13
4064,14
4087,75


In [44]:
totalgoaldf=summarize(group_by(totalgoals,team_api_id),totalGoals=sum(totalGoals))

In [45]:
totalgoaldf

team_api_id,totalGoals
1601,280
1773,126
1957,287
2033,138
2182,372
2183,184
2186,235
4049,35
4064,27
4087,180


In [46]:
Team=left_join(Team,totalgoaldf,by="team_api_id")
Team

team_api_id,team_long_name,team_short_name,sum_build,sum_chance,sum_defence,noOfPlayers,totalGoals
9987,KRC Genk,GEN,148,182,205,75,348
9993,Beerschot AC,BAC,110,185,210,72,174
10000,SV Zulte-Waregem,ZUL,147,153,200,87,306
9994,Sporting Lokeren,LOK,159,154,200,70,279
9984,KSV Cercle Brugge,CEB,145,200,195,64,213
8635,RSC Anderlecht,AND,149,182,190,76,427
9991,KAA Gent,GEN,142,184,166,88,390
9998,RAEC Mons,MON,160,168,138,46,129
7947,FCV Dender EH,DEN,NA,NA,NA,13,44
9985,Standard de Liège,STL,161,182,205,114,341


In [47]:
write.csv(Team, "team.csv",na="")